In [1]:
from pandora_bayesopt.test_functions.pest_control import PestControl, pest_control_price

In [2]:
import torch
from botorch.utils.sampling import draw_sobol_samples
from botorch.acquisition import ExpectedImprovement
from pandora_bayesopt.acquisition.gittins import GittinsIndex
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer

In [3]:
# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [4]:
seed = 0
torch.manual_seed(seed)

In [5]:
dim = 25
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
init_x = draw_sobol_samples(bounds=bounds, n=1, q=dim+1).squeeze(0)

In [6]:
def objective_function(X):
    choice_X = torch.floor(5*X)
    choice_X[choice_X == 5] = 4
    return PestControl(negate=True)(choice_X)

In [7]:
import numpy as np
     
def cost_function(X):
    choice_X = torch.floor(5*X)
    choice_X[choice_X == 5] = 4
    res = torch.stack([pest_control_price(x) for x in choice_X]).to(choice_X)
    # Add a small amount of noise to prevent training instabilities
    res += 1e-6 * torch.randn_like(res)
    return res 

In [8]:
init_x

tensor([[0.4751, 0.5925, 0.4945, 0.3139, 0.9561, 0.0833, 0.0988, 0.3475, 0.8469,
         0.7615, 0.8828, 0.8802, 0.8025, 0.8511, 0.3647, 0.9520, 0.2220, 0.6901,
         0.9973, 0.6273, 0.3472, 0.7656, 0.6587, 0.4764, 0.1585],
        [0.7977, 0.5988, 0.2493, 0.7424, 0.6684, 0.5202, 0.0243, 0.8263, 0.7274,
         0.2683, 0.0989, 0.6641, 0.1531, 0.0533, 0.8113, 0.8015, 0.6086, 0.3813,
         0.8859, 0.6664, 0.6712, 0.8236, 0.9889, 0.2629, 0.1451],
        [0.8653, 0.2530, 0.3578, 0.2487, 0.6613, 0.4413, 0.5199, 0.2375, 0.0419,
         0.6801, 0.6986, 0.7500, 0.5400, 0.8805, 0.8813, 0.4593, 0.5351, 0.8944,
         0.1966, 0.3889, 0.6816, 0.1965, 0.6133, 0.3215, 0.4993],
        [0.0672, 0.8454, 0.5426, 0.3719, 0.5108, 0.0282, 0.3609, 0.6418, 0.2187,
         0.2718, 0.8294, 0.5361, 0.4875, 0.9089, 0.7151, 0.6890, 0.1757, 0.5540,
         0.6684, 0.8071, 0.0187, 0.2317, 0.4319, 0.6631, 0.6238],
        [0.5548, 0.1569, 0.3566, 0.9287, 0.4057, 0.1726, 0.6179, 0.9103, 0.4576,
       

In [9]:
objective_function(init_x), cost_function(init_x)

(tensor([-16.8864, -17.2760, -18.7440, -18.8620, -18.8708, -17.6280, -18.1560,
         -19.2608, -18.5124, -19.1200, -18.2156, -18.0540, -18.0568, -17.5092,
         -17.9756, -17.8720, -18.5440, -19.8960, -16.7960, -19.1736, -17.0936,
         -18.0100, -19.2976, -17.5812, -19.5500, -18.3276]),
 tensor([15.1364, 13.4960, 16.0640, 15.1520, 15.1908, 13.4780, 16.1760, 13.6208,
         13.6724, 14.3500, 13.5856, 15.4240, 13.0268, 13.0192, 13.7256, 16.9320,
         15.5740, 13.6560, 14.8160, 13.5636, 12.7236, 12.6400, 15.7876, 13.3512,
         13.3500, 15.4576]))

In [10]:
Optimizer = BayesianOptimizer(
        objective=objective_function, 
        cost=cost_function,
        dim=dim, 
        maximize=True, 
        initial_points=init_x,
        input_standardize=True
    )

Optimizer.run_until_budget(
    budget=30, 
    acquisition_function_class = ExpectedImprovement
)

Iteration 0, New point: [0.19345925 0.51806268 0.91593191 0.26890095 0.71419715 0.68799047
 0.04760733 0.86441746 0.89524128 0.48543962 0.7819442  0.71727991
 0.50523438 0.86302971 0.81808607 0.44429197 0.71747928 0.85330233
 0.42631806 0.95584026 0.55825016 0.69168253 0.32468152 0.9428125
 0.06767279], New value: -16.29999731036551
Best observed value: -16.29999731036551
Current acquisition value: tensor(0.0880)
Cumulative cost: 14.320000744982508
Running time: 26.726480000000002

Iteration 1, New point: [0.16798882 0.50105485 0.88181964 0.27393041 0.71785274 0.64408442
 0.02619072 0.85315295 0.87466976 0.49492089 0.80328166 0.72529179
 0.54468276 0.86993695 0.80622825 0.47303263 0.68296665 0.83071156
 0.49429283 0.92005216 0.53367848 0.67332027 0.33835404 0.92445061
 0.06534654], New value: -16.16239992039369
Best observed value: -16.16239992039369
Current acquisition value: tensor(0.0801)
Cumulative cost: 28.532401194670726
Running time: 24.017742

Iteration 2, New point: [0.2148419

In [11]:
from pandora_bayesopt.acquisition.ei_puc import ExpectedImprovementWithCost
Optimizer = BayesianOptimizer(
        objective=objective_function, 
        cost=cost_function,
        dim=dim, 
        maximize=True, 
        initial_points=init_x,
        input_standardize=True
    )

Optimizer.run_until_budget(
    budget=30, 
    acquisition_function_class = ExpectedImprovementWithCost
)

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/optim/optimize.py:367: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)


Iteration 0, New point: [0.13939173 0.51626666 0.93377649 0.25597256 0.73141829 0.79620624
 0.0296328  0.9686591  0.88226664 0.46144682 0.78955547 0.68009312
 0.50351621 0.90017025 0.89132026 0.40435811 0.79714414 0.88582213
 0.36813966 0.99483831 0.59623346 0.67330878 0.19999782 0.9909474
 0.03359989], New value: -16.435600449472396
Best observed value: -16.435600449472396
Current acquisition value: tensor(0.0077)
Cumulative cost: 13.625600499752252
Running time: 18.695859999999996

Iteration 1, New point: [0.14754597 0.54019039 0.89995898 0.2546378  0.77918041 0.78548227
 0.05762605 0.96838533 0.88291491 0.48966642 0.7966716  0.68732446
 0.53608101 0.88886166 0.79964934 0.43230816 0.80991988 0.89319377
 0.36352891 0.97951368 0.6186219  0.68981198 0.15930404 0.97164719
 0.04608342], New value: -16.31280061994978
Best observed value: -16.31280061994978
Current acquisition value: tensor(0.0068)
Cumulative cost: 27.128399045327527
Running time: 4.983294999999998

Iteration 2, New point: 

In [12]:
Optimizer = BayesianOptimizer(
        objective=objective_function, 
        cost=cost_function,
        dim=dim, 
        maximize=True, 
        initial_points=init_x,
        input_standardize=True
    )

Optimizer.run_until_budget(
            budget = 30, 
            acquisition_function_class = GittinsIndex,
            lmbda = 0.0001
        )

/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/optim/optimize.py:367: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with statu

Iteration 0, New point: [0.70579026 0.61767867 0.45385434 0.6295011  0.8098968  0.28284879
 0.6343039  0.13856162 0.37136603 0.68614079 0.61376292 0.85597179
 0.41367375 0.64142898 0.92016916 0.30717739 0.70012623 0.81932064
 0.60235092 0.35661044 0.3078365  0.2824     0.3420966  0.47984517
 0.19992995], New value: -0.9299984843237176
Best observed value: -0.8700001203413739
Current acquisition value: tensor(1.4386)
Cumulative cost: 16.541200023005747
Running time: 72.3277



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/optim/optimize.py:367: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.'), OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with statu

Iteration 1, New point: [0.68616429 0.69681423 0.43278816 0.72611595 0.83875734 0.26457534
 0.55401027 0.18610637 0.30191294 0.61325178 0.55216271 0.88803815
 0.29551987 0.5639763  0.83915475 0.19783683 0.79959105 0.80725567
 0.62420954 0.25917003 0.19262629 0.26074367 0.22576351 0.42988147
 0.17876078], New value: -2.8399996717300993
Best observed value: -0.8700001203413739
Current acquisition value: tensor(1.6190)
Cumulative cost: 31.910801811247502
Running time: 72.40731299999999



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/optim/optimize.py:389: RuntimeWarning: Optimization failed on the second try, after generating a new set of initial conditions.
  warnings.warn(
